In [ ]:
import tensorflow as tf
print(tf.__version__)

# Validation, regularisation and callbacks

 ## Coding tutorials
 #### [1. Validation sets](#coding_tutorial_1)
 #### [2. Model regularisation](#coding_tutorial_2)
 #### [3. Introduction to callbacks](#coding_tutorial_3)
 #### [4. Early stopping / patience](#coding_tutorial_4)

***
<a id="coding_tutorial_1"></a>
## Validation sets

#### Load the data

In [ ]:
# Load the diabetes dataset
from sklearn.datasets import load_diabetes
diabaties_dataset = load_diabetes()
print(diabaties_dataset["DESCR"])

In [ ]:
diabaties_dataset.keys()

In [ ]:
# Save the input and target variables
data = diabaties_dataset['data']
target = diabaties_dataset['target']


In [ ]:
import numpy as np
# Normalise the target data (this will make clearer training curves)
target = (target - np.mean(target))/np.std(target)
target

In [ ]:
# Split the data into train and test sets
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(data,target,test_size=0.1 )
print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)
print(Y_test.shape)

#### Train a feedforward neural network model

In [ ]:
X_train.shape[1]

In [ ]:
# Build the model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def get_model():
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(128,activation='relu'),
        Dense(128,activation='relu'),
        Dense(128,activation='relu'),
        Dense(128,activation='relu'),
        Dense(128,activation='relu'),
        Dense(1,activation='linear')
    ])
    return model
model = get_model()

In [ ]:
# Print the model summary

model.summary()

In [ ]:
# Compile the model

model.compile(optimizer='adam',loss='mse', metrics=['mae'])

In [ ]:
# Train the model, with some of the data reserved for validation
history = model.fit(X_train,Y_train,validation_split=0.15,epochs=100,verbose=2,batch_size=64)


In [ ]:
# Evaluate the model on the test set
loss,mae = model.evaluate(X_test,Y_test)


#### Plot the learning curves

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Plot the training and validation loss

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

***
<a id="coding_tutorial_2"></a>
## Model regularisation

#### Adding regularisation with weight decay and dropout

In [ ]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers

In [ ]:
def get_regularised_model(wd, rate):
    model = Sequential([
        Dense(128, activation="relu", kernel_regularizer=regularizers.l2(wd), input_shape=(X_train.shape[1],)),
        Dropout(rate),
        Dense(128, activation="relu", kernel_regularizer=regularizers.l2(wd),),
        Dropout(rate),
        Dense(128, activation="relu", kernel_regularizer=regularizers.l2(wd),),
        Dropout(rate),
        Dense(128, activation="relu", kernel_regularizer=regularizers.l2(wd),),
        Dropout(rate),
        Dense(128, activation="relu", kernel_regularizer=regularizers.l2(wd),),
        Dropout(rate),
        Dense(128, activation="relu", kernel_regularizer=regularizers.l2(wd),),
        Dropout(rate),
        Dense(1)
    ])
    return model

In [ ]:
# Re-build the model with weight decay and dropout layers

model = get_regularised_model(0.01,0.05)

In [ ]:
# Compile the model

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# Train the model, with some of the data reserved for validation

history = model.fit(X_train, Y_train, validation_split=0.20, epochs=100, batch_size=2**7, verbose=False )

In [ ]:
# Evaluate the model on the test set
loss, mae = model.evaluate(X_test,Y_test, verbose=2)


#### Plot the learning curves

In [ ]:
# Plot the training and validation loss

import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

***
<a id="coding_tutorial_3"></a>
## Introduction to callbacks

#### Example training callback

In [ ]:
# Write a custom callback
from tensorflow.keras.callbacks import Callback



class myCustomTrainingCallBack(Callback):
    
    def on_train_begin(self, logs= None):
        print("Training Begin!! ")
        
    def on_train_batch_begin(self, batch, logs=None):
        print(f"Training Batch {batch} Begin")
    
    def on_train_batch_end(self, batch, logs=None):
        print(f"Training Batch {batch} Ended")
        
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Epoch #{epoch} Begin" )
        
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch #{epoch} End")
        
    def on_train_end(self,logs=None):
        print("Training Ended")
        
        
class myCustomTestingCallBack(Callback):
    
    def on_test_begin(self, logs= None):
        print("Testing Begin!! ")
        
    def on_test_batch_begin(self, batch, logs=None):
        print(f"Testing Batch {batch} Begin")
    
    def on_test_batch_end(self, batch, logs=None):
        print(f"Testing Batch {batch} Ended")
        
        
    def on_test_end(self,logs=None):
        print("Testing Ended")
        

class myCustomPredictionCallBack(Callback):
    
    def on_predict_begin(self, logs= None):
        print("Prediction Begin!! ")
        
    def on_predict_batch_begin(self, batch, logs=None):
        print(f"Prediction Batch {batch} Begin")
    
    def on_predict_batch_end(self, batch, logs=None):
        print(f"Predicton Batch {batch} Ended")
        
        
    def on_predict_end(self,logs=None):
        print("Precition Ended")

In [ ]:
# Re-build the model

model = get_regularised_model(1e-5, 0.3)


In [ ]:
# Compile the model
model.compile(optimizer='Adam', loss='mse')

#### Train the model with the callback

In [ ]:
# Train the model, with some of the data reserved for validation

history = model.fit(X_train,Y_train, epochs=5, batch_size=128, validation_split=0.1, callbacks=[myCustomTrainingCallBack()], verbose=0)

In [ ]:
# Evaluate the model

model.evaluate(X_test, Y_test, verbose=False, callbacks=[myCustomTestingCallBack()] )

In [ ]:
# Make predictions with the model

model.predict(X_test, verbose=False, callbacks=[myCustomPredictionCallBack()])

***
<a id="coding_tutorial_4"></a>
## Early stopping / patience

#### Re-train the models with early stopping

In [ ]:
# Re-train the unregularised model
unreg_model = get_model()
unreg_model.compile(optimizer='adam', loss='mse')
unreg_hist = unreg_model.fit(X_train,Y_train, epochs=100,
                             verbose=False,validation_split=0.2,
                             callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)] )


In [ ]:
# Evaluate the model on the test set
unreg_model.evaluate(X_test, Y_test)


In [ ]:
# Re-train the regularised model
regularized_model = get_regularised_model(1e-8,0.01)
regularized_model.compile(optimizer='adam', loss='mse')
regularized_hist = regularized_model.fit(X_train,Y_train, epochs=100,
                                         verbose=False,validation_split=0.2,
                                         callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)] )


In [ ]:
# Evaluate the model on the test set

regularized_model.evaluate(X_test,Y_test)

#### Plot the learning curves

In [ ]:
# Plot the training and validation loss

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 5))

fig.add_subplot(121)

plt.plot(unreg_hist.history['loss'])
plt.plot(unreg_hist.history['val_loss'])
plt.title('Unregularised model: loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

fig.add_subplot(122)

plt.plot(regularized_hist.history['loss'])
plt.plot(regularized_hist.history['val_loss'])
plt.title('Regularised model: loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

plt.show()